# Domain Adaptation Via Activation Shaping

**Teacher assistant** Iurada Leonardo

**Students**

- Bar Giorgio
- Distefano Giuseppe
- Incaviglia Salvatore

## Code

### Libraries

In [1]:
!pip install torch torchvision tqdm torchmetrics

import torch

import torch.backends.mps
from argparse import ArgumentParser

import os
import torchvision.transforms as T

from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CIFAR10

import numpy as np
import random
from PIL import Image

import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights

import torch.nn.functional as F
from torchmetrics import Accuracy
from tqdm import tqdm

import logging
import warnings

import argparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 9.5 MB/s eta 0:00:00


### Setup PACS dataset and environment

In [2]:
# Download PACS Dataset Images and Labels
!git clone https://github.com/MachineLearning2020/Homework3-PACS/
!git clone https://github.com/silvia1993/DANN_Template/

# Setup data
!rm -rf data || true
!rm -rf record || true
!mkdir data
!mkdir data/kfold
!cp -r Homework3-PACS/PACS/ data/kfold
!cp DANN_Template/txt_lists/*.txt data
!rm -rf Homework3-PACS/
!rm -rf DANN_Template/
!rm -rf sample_data || true

Cloning into 'Homework3-PACS'...
remote: Enumerating objects: 10032, done.
remote: Total 10032 (delta 0), reused 0 (delta 0), pack-reused 10032
Receiving objects: 100% (10032/10032), 174.13 MiB | 19.25 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (9993/9993), done.
Cloning into 'DANN_Template'...
remote: Enumerating objects: 23, done.
remote: Total 23 (delta 0), reused 0 (delta 0), pack-reused 23
Receiving objects: 100% (23/23), 33.86 KiB | 3.76 MiB/s, done.
Resolving deltas: 100% (5/5), done.


### Globals

#### globals

In [3]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

CONFIG = dotdict({})

if torch.cuda.is_available():
    CONFIG.device = 'cuda'
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    CONFIG.device = 'mps'
else:
    CONFIG.device = 'cpu'

CONFIG.dtype = torch.float32

#### parse args

In [4]:
def _clear_args(parsed_args):
    parsed_args.experiment_args = eval(parsed_args.experiment_args)
    parsed_args.dataset_args = eval(parsed_args.dataset_args)
    return parsed_args


def parse_arguments():
    parser = ArgumentParser()

    parser.add_argument('--seed', type=int, default=0, help='Seed used for deterministic behavior')
    parser.add_argument('--test_only', action='store_true', help='Whether to skip training')
    parser.add_argument('--cpu', action='store_true', help='Whether to force the usage of CPU')

    parser.add_argument('--experiment', type=str, default='baseline')
    parser.add_argument('--experiment_name', type=str, default='baseline')
    parser.add_argument('--experiment_args', type=str, default='{}')
    parser.add_argument('--dataset_args', type=str, default='{}')

    parser.add_argument('--batch_size', type=int, default=128)
    parser.add_argument('--epochs', type=int, default=30)
    parser.add_argument('--num_workers', type=int, default=5)
    parser.add_argument('--grad_accum_steps', type=int, default=1)

    return _clear_args(parser.parse_args())

#### Global variables

In [5]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_CLASSES = 7
BATCH_SIZE = 256
LR = 1e-3               # The initial Learning Rate
MOMENTUM = 0.9          # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5     # Regularization, you can keep this at the default
NUM_EPOCHS = 30         # Total number of training epochs (iterations over dataset)
STEP_SIZE = 20          # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1             # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

### Dataset

#### utils

In [6]:
class BaseDataset(Dataset):
    def __init__(self, examples, transform):
        self.examples = examples
        self.T = transform


    def __len__(self):
        return len(self.examples)


    def __getitem__(self, index):
        x, y = self.examples[index]
        x = Image.open(x).convert('RGB')
        x = self.T(x).to(CONFIG.dtype)
        y = torch.tensor(y).long()
        return x, y

######################################################

class DomainAdaptationDataset(Dataset):
    def __init__(self, source_examples, target_examples, transform):
        self.source_examples = source_examples
        self.target_examples = target_examples
        self.T = transform


    def __len__(self):
        return len(self.source_examples)


    def __getitem__(self, index):
        src_x, src_y = self.source_examples[index]
        src_x = Image.open(src_x).convert('RGB')
        src_x = self.T(src_x).to(CONFIG.dtype)
        src_y = torch.tensor(src_y).long()

        # randomly sample 'target_examples' to obtain targ_x
        target_idx = torch.randint(len(self.target_examples), size=(1,)).item()
        targ_x, _ = self.target_examples[target_idx]
        targ_x = Image.open(targ_x).convert('RGB')
        targ_x = self.T(targ_x).to(CONFIG.dtype)

        return src_x, src_y, targ_x

######################################################

# [OPTIONAL] TODO: modify 'BaseDataset' for the Domain Generalization setting.
# Hint: combine the examples from the 3 source domains into a single 'examples' list
#class DomainGeneralizationDataset(Dataset):
#    def __init__(self, examples, transform):
#        self.examples = examples
#        self.T = transform
#
#    def __len__(self):
#        return len(self.examples)
#
#    def __getitem__(self, index):
#        x1, x2, x3 = self.examples[index]
#        x1, x2, x3 = self.T(x1), self.T(x2), self.T(x3)
#        targ_x = self.T(targ_x)
#        return x1, x2, x3

######################################################

class SeededDataLoader(DataLoader):
    def __init__(self, dataset: Dataset, batch_size=1, shuffle=None,
                 sampler=None,
                 batch_sampler=None,
                 num_workers=0, collate_fn=None,
                 pin_memory=False, drop_last=False, timeout=0,
                 worker_init_fn=None, multiprocessing_context=None,
                 generator=None, *, prefetch_factor=None, persistent_workers=False,
                 pin_memory_device=""):

        if not CONFIG.use_nondeterministic:
            def seed_worker(worker_id):
                worker_seed = torch.initial_seed() % 2**32
                np.random.seed(worker_seed)
                random.seed(worker_seed)

            generator = torch.Generator()
            generator.manual_seed(CONFIG.seed)

            worker_init_fn = seed_worker

        super().__init__(dataset, batch_size, shuffle, sampler, batch_sampler, num_workers, collate_fn,
                         pin_memory, drop_last, timeout, worker_init_fn, multiprocessing_context, generator,
                         prefetch_factor=prefetch_factor, persistent_workers=persistent_workers,
                         pin_memory_device=pin_memory_device)

#### PACS

In [7]:
def get_transform(size, mean, std, preprocess):
    transform = []
    if preprocess:
        transform.append(T.Resize(256))
        transform.append(T.RandomResizedCrop(size=size, scale=(0.7, 1.0)))
        transform.append(T.RandomHorizontalFlip())
    else:
        transform.append(T.Resize(size))
    transform.append(T.ToTensor())
    transform.append(T.Normalize(mean, std))
    return T.Compose(transform)


def load_data():
    CONFIG.num_classes = 7
    CONFIG.data_input_size = (3, 224, 224)

    # Create transforms
    mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225) # ImageNet Pretrain statistics
    train_transform = get_transform(size=224, mean=mean, std=std, preprocess=True)
    test_transform = get_transform(size=224, mean=mean, std=std, preprocess=False)

    # Load examples
    source_examples, target_examples = [], []
    if CONFIG.experiment in ['baseline', 'random', 'domain_adaptation','extension2_1_2','extension2_1_3','extension2_2_2','extension2_2_3']:
        # Load source
        f = open(os.path.join(CONFIG.dataset_args['text_root'], f"{CONFIG.dataset_args['source_domain']}.txt"), 'r')
        for line in f:
            line = line.strip().split()
            path, label = line[0].split('/')[0:], int(line[1])
            source_examples.append((os.path.join(CONFIG.dataset_args['images_root'], *path), label))
        f.close()

        # Load target
        f = open(os.path.join(CONFIG.dataset_args['text_root'], f"{CONFIG.dataset_args['target_domain']}.txt"), 'r')
        for line in f:
            line = line.strip().split()
            path, label = line[0].split('/')[0:], int(line[1])
            target_examples.append((os.path.join(CONFIG.dataset_args['images_root'], *path), label))
        f.close()

    # create Dataset object for the baseline and the Random Maps Ablation experiments
    if CONFIG.experiment in ['baseline', 'random','extension2_1_2','extension2_2_2']:
        train_dataset = BaseDataset(source_examples, transform=train_transform)
        test_dataset = BaseDataset(target_examples, transform=test_transform)

    # create Dataset object for the Domain Adaptation experiments
    if CONFIG.experiment in ['domain_adaptation','extension2_1_3','extension2_2_3']:
        train_dataset = DomainAdaptationDataset(source_examples, target_examples, transform=train_transform)
        test_dataset = BaseDataset(target_examples, transform=test_transform)

    ######################################################

    # Dataloaders
    train_loader = SeededDataLoader(
        train_dataset,
        batch_size=CONFIG.batch_size,
        shuffle=True,
        num_workers=CONFIG.num_workers,
        pin_memory=True,
        persistent_workers=True
    )

    test_loader = SeededDataLoader(
        test_dataset,
        batch_size=CONFIG.batch_size,
        shuffle=False,
        num_workers=CONFIG.num_workers,
        pin_memory=True,
        persistent_workers=True
    )

    return {'train': train_loader, 'test': test_loader}

### Models

#### Base ResNet 18

In [8]:
class BaseResNet18(nn.Module):
    def __init__(self):
        super(BaseResNet18, self).__init__()
        self.resnet = resnet18(weights=ResNet18_Weights)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 7)


    def forward(self, x):
        return self.resnet(x)

#### ASH ResNet 18

In [9]:
# 'BaseResNet18' including the Activation Shaping Module
class ASHResNet18(nn.Module):
    def __init__(self):
        super(ASHResNet18, self).__init__()
        self.resnet = resnet18(weights=ResNet18_Weights)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 7)

        self.hook_handles = {}

    def get_random_activation_map_hook(self, mask_out_ratio):

        # Activation Shaping Module as a function that shall be hooked via 'register_forward_hook'
        # The hook captures the mask_out_ratio from the parent scope
        def random_activation_map_hook(module, input, output):
            # create a mask tensor with a given ratio of zeros
            print('mask_out_ratio: ', mask_out_ratio)
            rand_mat = torch.rand_like(output)
            mask = torch.where(rand_mat <= mask_out_ratio, 0.0, 1.0)
            print('random_mask sum(): ', mask.sum())

            # binarize both activation map and mask using zero as threshold
            A_binary = torch.where(output<=0, 0.0, 1.0)
            M_binary = torch.where(mask<=0, 0.0, 1.0)

            # return the element-wise product of activation map and mask
            shaped_output = A_binary * M_binary
            return shaped_output

        return random_activation_map_hook


    def register_random_activation_maps_hooks(self, module_placement, mask_out_ratio):
        for layer_name, module in self.resnet.named_modules():
            # if (isinstance(module, nn.Conv2d) and layer_name in module_placement):
            if ((isinstance(module, nn.ReLU) or isinstance(module, nn.Conv2d)) and layer_name in module_placement):
                print('Register a hook to perform Random Maps Ablation on layer ', layer_name, module)
                hook = self.get_random_activation_map_hook(mask_out_ratio)
                self.hook_handles[layer_name] = module.register_forward_hook(hook)


    def remove_random_activation_maps_hooks(self):
        for layer_name, handle in self.hook_handles.items():
            print('Remove the hook used to perform Activation Shaping on layer ', layer_name)
            handle.remove()


    def forward(self, x):
        return self.resnet(x)

#### ResNet 18 for Domain Adaptation

In [10]:
# 'BaseResNet18' including the Activation Shaping Module for Domain Adaptation
class DAResNet18(nn.Module):
    def __init__(self, module_placement=None):
        super(DAResNet18, self).__init__()
        self.resnet = resnet18(weights=ResNet18_Weights)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 7)

        self.activation_maps = {}
        self.activation_map_hook_handles = {}
        self.activation_shaping_hook_handles = {}

    def get_extract_activation_map_hook (self, layer_name):
        def extract_activation_map_hook(module, input, output):
            self.activation_maps[layer_name] = output.detach().clone()
        return extract_activation_map_hook

    def get_activation_shaping_hook (self, mask):
        # Activation Shaping Module as a function that shall be hooked via 'register_forward_hook'
        # The hook captures mask variable from the parent scope

        def activation_shaping_hook (module, input, output):

            # binarize both activation map and mask using zero as threshold
            A_binary = torch.where(output <= 0, 0.0, 1.0)
            M_binary = torch.where(mask <= 0, 0.0, 1.0)

            # return the element-wise product of activation map and mask
            shaped_output = A_binary * M_binary
            return shaped_output

        return activation_shaping_hook


    def register_extract_activation_map_hooks (self, module_placement):
        # Register hook(s) (1st hook) to store activation map
        for layer_name, module in self.resnet.named_modules():
            if (isinstance(module, nn.Conv2d) and layer_name in module_placement):
                hook = self.get_extract_activation_map_hook(layer_name)
                self.activation_map_hook_handles[layer_name] = module.register_forward_hook(hook)


    def remove_extract_activation_map_hooks (self):
        # Remove hook(s) used to store activation map
        for layer_name, handle in self.activation_map_hook_handles.items():
            handle.remove()


    def register_activation_shaping_hooks (self, module_placement):
        # Register the Activation Shaping Module hook(s) (2nd hook)
        for layer_name, module in self.resnet.named_modules():
            if (isinstance(module, nn.Conv2d) and layer_name in self.activation_maps):
                hook = self.get_activation_shaping_hook(self.activation_maps[layer_name])
                self.activation_shaping_hook_handles[layer_name] = module.register_forward_hook(hook)


    def remove_activation_shaping_hooks (self):
        # Remove hook(s) used to perform Activation Shaping
        for layer_name, handle in self.activation_shaping_hook_handles.items():
            handle.remove()


    def forward(self, x):
        return self.resnet(x)

#### ResNet 18 for ablation studies

##### 1.2

In [11]:
# 'BaseResNet18' including the Activation Shaping Module
class Abl12ResNet18(nn.Module):
    def __init__(self):
        super(Abl12ResNet18, self).__init__()
        self.resnet = resnet18(weights=ResNet18_Weights)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 7)
        self.hook_handles = {}


    def get_random_activation_map_hook(self, mask_out_ratio):
        # Activation Shaping Module as a function that shall be hooked via 'register_forward_hook'
        # The hook captures the mask_out_ratio from the parent scope
        def random_activation_map_hook(module, input, output):
            # create a mask tensor with a given ratio of zeros
            rand_mat = torch.rand_like(output)
            mask = torch.where(rand_mat <= mask_out_ratio, 0.0, 1.0)
            print('random_mask sum(): ', mask.sum())

            # binarize both activation map and mask using zero as threshold
            A_binary = torch.tensor(output)
            M_binary = torch.tensor(mask)

            # return the element-wise product of activation map and mask
            shaped_output = A_binary * M_binary
            return shaped_output

        return random_activation_map_hook


    def register_random_activation_maps_hooks(self, module_placement, mask_out_ratio):
        for layer_name, module in self.resnet.named_modules():
            if (isinstance(module, nn.Conv2d) and layer_name in module_placement):
                print('Register a hook to perform Random Maps Ablation on layer ', layer_name, module)
                hook = self.get_random_activation_map_hook(mask_out_ratio)
                self.hook_handles[layer_name] = module.register_forward_hook(hook)


    def remove_random_activation_maps_hooks(self):
        for layer_name, handle in self.hook_handles.items():
            print('Remove the hook used to perform Activation Shaping on layer ', layer_name)
            handle.remove()


    def forward(self, x):
        return self.resnet(x)

##### 1.3

In [12]:
# 'BaseResNet18' including the Activation Shaping Module for Domain Adaptation--->Punto3
class Abl13ResNet18(nn.Module):
    def __init__(self, module_placement=None):
        super(Abl13ResNet18, self).__init__()
        self.resnet = resnet18(weights=ResNet18_Weights)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 7)
        self.activation_maps = {}
        self.activation_map_hook_handles = {}
        self.activation_shaping_hook_handles = {}


    def get_extract_activation_map_hook (self, layer_name):
        def extract_activation_map_hook(module, input, output):
            self.activation_maps[layer_name] = output.detach().clone()
        return extract_activation_map_hook


    def get_activation_shaping_hook (self, mask):
        # Activation Shaping Module as a function that shall be hooked via 'register_forward_hook'
        def activation_shaping_hook (module, input, output):
            A_binary = torch.where(output <= 0, torch.tensor(0.0), torch.tensor(1.0))
            A = torch.tensor(output)
            M = torch.tensor(mask)
            shaped_output = A* M
            return shaped_output
        return activation_shaping_hook


    def register_extract_activation_map_hooks (self, module_placement):
        # Register hook(s) (1st hook) to store activation map
        for layer_name, module in self.resnet.named_modules():
            if (isinstance(module, nn.Conv2d) and layer_name in module_placement):
                hook = self.get_extract_activation_map_hook(layer_name)
                self.activation_map_hook_handles[layer_name] = module.register_forward_hook(hook)


    def remove_extract_activation_map_hooks (self):
        # Remove hook(s) used to store activation map
        for layer_name, handle in self.activation_map_hook_handles.items():
            handle.remove()


    def register_activation_shaping_hooks (self, module_placement):
        # Register the Activation Shaping Module hook(s) (2nd hook)
        for layer_name, module in self.resnet.named_modules():
            if (isinstance(module, nn.Conv2d) and layer_name in module_placement):
                hook = self.get_activation_shaping_hook(self.activation_maps[layer_name])
                self.activation_shaping_hook_handles[layer_name] = module.register_forward_hook(hook)


    def remove_activation_shaping_hooks (self):
        # Remove hook(s) used to perform Activation Shaping
        for layer_name, handle in self.activation_shaping_hook_handles.items():
            handle.remove()


    def forward(self, x):
        return self.resnet(x)

##### 2.2

In [13]:
def get_activation_shaping_hook_extension2_2_2(mask, topk):
    def activation_shaping_hook(module, input, output):
        M = torch.tensor(mask)
        A = torch.tensor(output)
        # Calcola i top-k values di A
        topk_values,top_indices = torch.topk(A,4)
        # Crea una maschera per gli elementi in topk
        M_binary= torch.where(M >= topk_values.min(), torch.tensor(1.0), torch.tensor(0.0))
        # Moltiplica A per M
        shaped_output = A * M_binary

        return shaped_output
    return activation_shaping_hook

######################################################

# 'BaseResNet18' including the Activation Shaping Module
class Abl22ResNet18(nn.Module):
    def __init__(self):
        super(Abl22ResNet18, self).__init__()
        self.resnet = resnet18(weights=ResNet18_Weights)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 7)
        self.hook_handles = {}


    def get_random_activation_map_hook(self, mask_out_ratio):
        def activation_shaping_hook(module, input, output):
            rand_mat = torch.rand_like(output)
            mask = torch.where(rand_mat <= mask_out_ratio, 0.0, 1.0)
            M = torch.tensor(mask)
            A = torch.tensor(output)
            # Calcola i topk values di A
            topk_values,top_indices = torch.topk(A,4)
            # Crea una maschera per gli elementi in topk
            M_binary= torch.where(M >= topk_values.min(), torch.tensor(1.0), torch.tensor(0.0))
            # Moltiplica A per M
            shaped_output = A * M_binary

            return shaped_output
        return activation_shaping_hook


    def register_random_activation_maps_hooks(self, module_placement, mask_out_ratio):
        for layer_name, module in self.resnet.named_modules():
            if (isinstance(module, nn.Conv2d) and layer_name in module_placement):
                print('Register a hook to perform Random Maps Ablation on layer ', layer_name, module)
                hook = self.get_random_activation_map_hook(mask_out_ratio)
                self.hook_handles[layer_name] = module.register_forward_hook(hook)


    def remove_random_activation_maps_hooks(self):
        for layer_name, handle in self.hook_handles.items():
            print('Remove the hook used to perform Activation Shaping on layer ', layer_name)
            handle.remove()


    def forward(self, x):
        return self.resnet(x)

##### 2.3

In [14]:
# 'BaseResNet18' including the Activation Shaping Module for Domain Adaptation--->Punto3
class Abl23ResNet18(nn.Module):
    def __init__(self, module_placement=None):
        super(Abl23ResNet18, self).__init__()
        self.resnet = resnet18(weights=ResNet18_Weights)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 7)
        self.activation_maps = {}
        self.activation_map_hook_handles = {}
        self.activation_shaping_hook_handles = {}


    def get_extract_activation_map_hook (self, layer_name):
        def extract_activation_map_hook(module, input, output):
            self.activation_maps[layer_name] = output.detach().clone()
        return extract_activation_map_hook


    def get_activation_shaping_hook (self, mask):
        # Activation Shaping Module as a function that shall be hooked via 'register_forward_hook'
        def activation_shaping_hook (module, input, output):
            M = torch.where(mask <= 0, torch.tensor(0.0), torch.tensor(1.0))
            A = torch.tensor(output)
            #topk_values, _ = torch.topk(A.view(-1), topk)
            topk_values, _ = torch.topk(A.view(-1), 4)
            topk_mask = torch.where(A >= topk_values.min(), torch.tensor(1.0), torch.tensor(0.0))
            M *= topk_mask.view_as(M)
            shaped_output = A * M

            return shaped_output
        return activation_shaping_hook


    def register_extract_activation_map_hooks (self, module_placement):
        # Register hook(s) (1st hook) to store activation map
        for layer_name, module in self.resnet.named_modules():
            if (isinstance(module, nn.Conv2d) and layer_name in module_placement):
                hook = self.get_extract_activation_map_hook(layer_name)
                self.activation_map_hook_handles[layer_name] = module.register_forward_hook(hook)


    def remove_extract_activation_map_hooks (self):
        # Remove hook(s) used to store activation map
        for layer_name, handle in self.activation_map_hook_handles.items():
            handle.remove()


    def register_activation_shaping_hooks (self, module_placement, target_activation_maps):
        # Register the Activation Shaping Module hook(s) (2nd hook)
        for layer_name, module in self.resnet.named_modules():
            if (isinstance(module, nn.Conv2d) and layer_name in module_placement):
                hook = self.get_activation_shaping_hook(target_activation_maps[layer_name])
                self.activation_shaping_hook_handles[layer_name] = module.register_forward_hook(hook)


    def remove_activation_shaping_hooks (self):
        # Remove hook(s) used to perform Activation Shaping
        for layer_name, handle in self.activation_shaping_hook_handles.items():
            handle.remove()


    def forward(self, x):
        return self.resnet(x)

### Run

#### Evaluate

In [15]:
@torch.no_grad()
def evaluate(model, data):
    model.eval()

    acc_meter = Accuracy(task='multiclass', num_classes=CONFIG.num_classes)
    acc_meter = acc_meter.to(CONFIG.device)

    loss = [0.0, 0]
    for x, y in tqdm(data):
        with torch.autocast(device_type=CONFIG.device, dtype=torch.float16, enabled=True):
            x, y = x.to(CONFIG.device), y.to(CONFIG.device)
            logits = model(x)
            acc_meter.update(logits, y)
            loss[0] += F.cross_entropy(logits, y).item()
            loss[1] += x.size(0)

    accuracy = acc_meter.compute()
    loss = loss[0] / loss[1]
    logging.info(f'Accuracy: {100 * accuracy:.2f} - Loss: {loss}')
    print(f'Accuracy: {100 * accuracy:.2f} - Loss: {loss} \n')

#### Train

In [16]:
def train(model, data):
    # Create optimizers & schedulers
    optimizer = torch.optim.SGD(model.parameters(), weight_decay=0.0005, momentum=0.9, nesterov=True, lr=0.001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=int(CONFIG.epochs * 0.8), gamma=0.1)
    scaler = torch.cuda.amp.GradScaler(enabled=True)

    # Load checkpoint (if it exists)
    cur_epoch = 0
    if os.path.exists(os.path.join('record', CONFIG.experiment_name, 'last.pth')):
        checkpoint = torch.load(os.path.join('record', CONFIG.experiment_name, 'last.pth'))
        cur_epoch = checkpoint['epoch']
        optimizer.load_state_dict(checkpoint['optimizer'])
        scheduler.load_state_dict(checkpoint['scheduler'])
        model.load_state_dict(checkpoint['model'])

    # Optimization loop
    for epoch in range(cur_epoch, CONFIG.epochs):
        model.train()

        for batch_idx, batch in enumerate(tqdm(data['train'])):

            # use a separate invocation of autocast for every forward pass.
            if CONFIG.experiment in ['domain_adaptation']:
                with torch.autocast(device_type=CONFIG.device, dtype=torch.float16, enabled=True):
                    model.eval()
                    # Register hook(s) to store the target domain activation maps and perform a forward pass
                    src_x, src_y, targ_x = batch
                    targ_x = targ_x.to(CONFIG.device)
                    module_placement = CONFIG.experiment_args['module_placement']
                    model.register_extract_activation_map_hooks(module_placement)
                    with torch.no_grad():
                        model(targ_x)
                    model.remove_extract_activation_map_hooks()
                    model.train()
            elif CONFIG.experiment in ['extension2_1_3']:
                with torch.autocast(device_type=CONFIG.device, dtype=torch.float16, enabled=True):
                    model.eval()
                    # Register hook(s) to store the target domain activation maps and perform a forward pass
                    src_x, src_y, targ_x = batch
                    targ_x = targ_x.to(CONFIG.device)
                    module_placement = CONFIG.experiment_args['module_placement']
                    model.register_extract_activation_map_hooks(module_placement)
                    with torch.no_grad():
                        model(targ_x)
                    model.remove_extract_activation_map_hooks()
                    model.train()
            elif CONFIG.experiment in ['extension2_2_3']:
                with torch.autocast(device_type=CONFIG.device, dtype=torch.float16, enabled=True):
                    model.eval()
                    # Register hook(s) to store the target domain activation maps and perform a forward pass
                    src_x, src_y, targ_x = batch
                    targ_x = targ_x.to(CONFIG.device)
                    module_placement = CONFIG.experiment_args['module_placement']
                    model.register_extract_activation_map_hooks(module_placement)
                    with torch.no_grad():
                        model(targ_x)
                    model.remove_extract_activation_map_hooks()
                    model.train()

            # Compute loss
            with torch.autocast(device_type=CONFIG.device, dtype=torch.float16, enabled=True):

                if CONFIG.experiment in ['baseline']:
                    x, y = batch
                    x, y = x.to(CONFIG.device), y.to(CONFIG.device)
                    loss = F.cross_entropy(model(x), y)
                elif CONFIG.experiment in ['random']:
                    x, y = batch
                    x, y = x.to(CONFIG.device), y.to(CONFIG.device)
                    # Randomly turn off some outputs of each layer
                    module_placement = CONFIG.experiment_args['module_placement']
                    mask_out_ratio = CONFIG.experiment_args['mask_out_ratio']
                    model.register_random_activation_maps_hooks(module_placement, mask_out_ratio)
                    loss = F.cross_entropy(model(x), y)
                    model.remove_random_activation_maps_hooks()
                elif CONFIG.experiment in ['extension2_1_2']:
                    x, y = batch
                    x, y = x.to(CONFIG.device), y.to(CONFIG.device)
                    # Randomly turn off some outputs of each layer
                    module_placement = CONFIG.experiment_args['module_placement']
                    mask_out_ratio = CONFIG.experiment_args['mask_out_ratio']
                    model.register_random_activation_maps_hooks(module_placement, mask_out_ratio)
                    loss = F.cross_entropy(model(x), y)
                    model.remove_random_activation_maps_hooks()
                elif CONFIG.experiment in ['extension2_2_2']:
                    x, y = batch
                    x, y = x.to(CONFIG.device), y.to(CONFIG.device)
                    # Randomly turn off some outputs of each layer
                    module_placement = CONFIG.experiment_args['module_placement']
                    mask_out_ratio = CONFIG.experiment_args['mask_out_ratio']
                    model.register_random_activation_maps_hooks(module_placement, mask_out_ratio)
                    loss = F.cross_entropy(model(x), y)
                    model.remove_random_activation_maps_hooks()
                elif CONFIG.experiment in ['domain_adaptation']:
                    src_x, src_y, targ_x = batch
                    src_x, src_y = src_x.to(CONFIG.device), src_y.to(CONFIG.device)
                    module_placement = CONFIG.experiment_args['module_placement']
                    # Register the Activation Shaping hook(s) and perform a forward pass (source domain images)
                    model.register_activation_shaping_hooks(module_placement)
                    loss = F.cross_entropy(model(src_x), src_y)
                    model.remove_activation_shaping_hooks()
                elif CONFIG.experiment in ['extension2_1_3']:
                    src_x, src_y, targ_x = batch
                    src_x, src_y = src_x.to(CONFIG.device), src_y.to(CONFIG.device)
                    module_placement = CONFIG.experiment_args['module_placement']
                    # Register the Activation Shaping hook(s) and perform a forward pass (source domain images)
                    model.register_activation_shaping_hooks(module_placement)
                    loss = F.cross_entropy(model(src_x), src_y)
                    model.remove_activation_shaping_hooks()
                elif CONFIG.experiment in ['extension2_2_3']:
                    src_x, src_y, targ_x = batch
                    src_x, src_y = src_x.to(CONFIG.device), src_y.to(CONFIG.device)
                    module_placement = CONFIG.experiment_args['module_placement']
                    # Register the Activation Shaping hook(s) and perform a forward pass (source domain images)
                    model.register_activation_shaping_hooks(module_placement)
                    loss = F.cross_entropy(model(src_x), src_y)
                    model.remove_activation_shaping_hooks()

            # Optimization step
            scaler.scale(loss / CONFIG.grad_accum_steps).backward()

            if ((batch_idx + 1) % CONFIG.grad_accum_steps == 0) or (batch_idx + 1 == len(data['train'])):
                scaler.step(optimizer)
                optimizer.zero_grad(set_to_none=True)
                scaler.update()

        scheduler.step()

        # Test current epoch
        logging.info(f'[TEST @ Epoch={epoch}]')
        evaluate(model, data['test'])

        # Save checkpoint
        checkpoint = {
            'epoch': epoch + 1,
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict(),
            'model': model.state_dict()
        }
        torch.save(checkpoint, os.path.join('record', CONFIG.experiment_name, 'last.pth'))

#### Main

In [18]:
'''
layer_output_shapes = {
    'layer1.0.conv1': (64, 56, 56),         'layer1.0.relu': (64, 56, 56),          'layer1.0.conv2': (64, 56, 56),
    'layer1.1.conv1': (64, 56, 56),         'layer1.1.bn1': (64, 56, 56),           'layer1.1.relu': (64, 56, 56),          'layer1.1.conv2': (64, 56, 56),         'layer1.1.bn2': (64, 56, 56),
    'layer2.0.conv1': (128, 28, 28),        'layer2.0.relu': (128, 28, 28),         'layer2.0.conv2': (128, 28, 28),
    'layer2.1.conv1': (128, 28, 28),        'layer2.1.relu': (128, 28, 28),         'layer2.1.conv2': (128, 28, 28),
    'layer3.0.conv1': (256, 14, 14),        'layer3.0.relu': (256, 14, 14),         'layer3.0.conv2': (256, 14, 14),
    'layer3.1.conv1': (256, 14, 14),        'layer3.1.relu': (256, 14, 14),         'layer3.1.conv2': (256, 14, 14),
    'layer4.0.conv1': (512, 7, 7),          'layer4.0.relu': (512, 7, 7),           'layer4.0.conv2': (512, 7, 7),
    'layer4.1.conv1': (512, 7, 7),          'layer4.1.relu': (512, 7, 7),           'layer4.1.conv2': (512, 7, 7)
}

def create_random_mask(layer_name = 'layer4.1.conv2', mask_out_ratio = 0.0):
    # create a mask tensor with a given ratio of zeros
    layer_output_shape = layer_output_shapes[layer_name]
    rand_mat = torch.rand(layer_output_shape).to(CONFIG.device)
    mask = torch.where(rand_mat <= mask_out_ratio, 0.0, 1.0).to(CONFIG.device)
    return mask
'''

"\nlayer_output_shapes = {\n    'layer1.0.conv1': (64, 56, 56),         'layer1.0.relu': (64, 56, 56),          'layer1.0.conv2': (64, 56, 56),\n    'layer1.1.conv1': (64, 56, 56),         'layer1.1.bn1': (64, 56, 56),           'layer1.1.relu': (64, 56, 56),          'layer1.1.conv2': (64, 56, 56),         'layer1.1.bn2': (64, 56, 56),\n    'layer2.0.conv1': (128, 28, 28),        'layer2.0.relu': (128, 28, 28),         'layer2.0.conv2': (128, 28, 28),\n    'layer2.1.conv1': (128, 28, 28),        'layer2.1.relu': (128, 28, 28),         'layer2.1.conv2': (128, 28, 28),\n    'layer3.0.conv1': (256, 14, 14),        'layer3.0.relu': (256, 14, 14),         'layer3.0.conv2': (256, 14, 14),\n    'layer3.1.conv1': (256, 14, 14),        'layer3.1.relu': (256, 14, 14),         'layer3.1.conv2': (256, 14, 14),\n    'layer4.0.conv1': (512, 7, 7),          'layer4.0.relu': (512, 7, 7),           'layer4.0.conv2': (512, 7, 7),\n    'layer4.1.conv1': (512, 7, 7),          'layer4.1.relu': (512, 7, 7)

In [19]:
def main ():
    # Load dataset
    data = load_data()

    # Load model
    if CONFIG.experiment in ['baseline']:
        model = BaseResNet18()

    elif CONFIG.experiment in ['random']:
        model = ASHResNet18()
    elif CONFIG.experiment in ['extension2_1_2']:
        model = Abl12ResNet18()
    elif CONFIG.experiment in ['extension2_2_2']:
        model = Abl22ResNet18()

    elif CONFIG.experiment in ['domain_adaptation']:
         module_placement = CONFIG.experiment_args['module_placement']
         model= DAResNet18(module_placement)
    elif CONFIG.experiment in ['extension2_1_3']:
         module_placement = CONFIG.experiment_args['module_placement']
         model= Abl13ResNet18(module_placement)
    elif CONFIG.experiment in ['extension2_2_3']:
         module_placement = CONFIG.experiment_args['module_placement']
         model= Abl23ResNet18(module_placement)

    ######################################################

    model.to(CONFIG.device)

    if not CONFIG.test_only:
        train(model, data)
    else:
        evaluate(model, data['test'])


### Custom experiment

#### Configurations

In [20]:
configurations = {
    ##### 0 - BASELINE #####
    "0.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'baseline',
        'experiment_name': '0.1',
        'experiment_args': '{}',
        'dataset_args': { 'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "0.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'baseline',
        'experiment_name': '0.2',
        'experiment_args': '{}',
        'dataset_args': { 'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "0.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'baseline',
        'experiment_name': '0.3',
        'experiment_args': '{}',
        'dataset_args': { 'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },

    ##### 1 - ACTIVATION SHAPING MODULE #####
    #### 1.1 - ONE ASH MODULE ####
    ### 1.1.1 - ASH MODULE AFTER layer1.0.conv1 ###
    "1.1.1.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.1.1',
        'experiment_args': {'module_placement': ['layer1.0.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.1.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.1.2',
        'experiment_args': {'module_placement': ['layer1.0.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.1.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.1.3',
        'experiment_args': {'module_placement': ['layer1.0.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    ### 1.1.2 - ASH MODULE AFTER layer1.0.conv2 ###
    "1.1.2.2" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.2.2',
        'experiment_args': {'module_placement': ['layer1.0.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.2.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.2.2',
        'experiment_args': {'module_placement': ['layer1.0.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.2.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.2.3',
        'experiment_args': {'module_placement': ['layer1.0.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    ### 1.1.3 - ASH MODULE AFTER layer1.1.conv1 ###
    "1.1.3.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.3.1',
        'experiment_args': {'module_placement': ['layer1.1.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.3.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.3.2',
        'experiment_args': {'module_placement': ['layer1.1.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.3.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.3.3',
        'experiment_args': {'module_placement': ['layer1.1.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    ### 1.1.4 - ASH MODULE AFTER layer1.1.conv2 ###
    "1.1.4.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.4.1',
        'experiment_args': {'module_placement': ['layer1.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.4.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.4.2',
        'experiment_args': {'module_placement': ['layer1.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.4.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.4.3',
        'experiment_args': {'module_placement': ['layer1.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    ### 1.1.5 - ASH MODULE AFTER layer2.0.conv1 ###
    "1.1.5.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.5.1',
        'experiment_args': {'module_placement': ['layer2.0.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.5.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.5.2',
        'experiment_args': {'module_placement': ['layer2.0.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.5.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.5.3',
        'experiment_args': {'module_placement': ['layer2.0.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    ### 1.1.6 - ASH MODULE AFTER layer2.0.conv2 ###
    "1.1.6.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.6.1',
        'experiment_args': {'module_placement': ['layer2.0.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.6.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.6.2',
        'experiment_args': {'module_placement': ['layer2.0.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.6.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.6.3',
        'experiment_args': {'module_placement': ['layer2.0.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    ### 1.1.7 - ASH MODULE AFTER layer2.1.conv1 ###
    "1.1.7.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.7.1',
        'experiment_args': {'module_placement': ['layer2.1.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.7.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.7.2',
        'experiment_args': {'module_placement': ['layer2.1.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.7.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.7.3',
        'experiment_args': {'module_placement': ['layer2.1.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    ### 1.1.8 - ASH MODULE AFTER layer2.1.conv2 ###
    "1.1.8.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.8.1',
        'experiment_args': {'module_placement': ['layer2.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.8.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.8.2',
        'experiment_args': {'module_placement': ['layer2.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.8.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.8.3',
        'experiment_args': {'module_placement': ['layer2.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    ### 1.1.9 - ASH MODULE AFTER layer3.0.conv1 ###
    "1.1.9.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.9.1',
        'experiment_args': {'module_placement': ['layer3.0.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.9.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.9.2',
        'experiment_args': {'module_placement': ['layer3.0.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.9.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.9.3',
        'experiment_args': {'module_placement': ['layer3.0.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    ### 1.1.10 - ASH MODULE AFTER layer3.0.conv2 ###
    "1.1.10.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.10.1',
        'experiment_args': {'module_placement': ['layer3.0.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.10.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.10.2',
        'experiment_args': {'module_placement': ['layer3.0.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.10.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.10.3',
        'experiment_args': {'module_placement': ['layer3.0.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    ### 1.1.11 - ASH MODULE AFTER layer3.1.conv1 ###
    "1.1.11.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.11.1',
        'experiment_args': {'module_placement': ['layer3.1.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.11.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.11.2',
        'experiment_args': {'module_placement': ['layer3.1.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.11.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.11.3',
        'experiment_args': {'module_placement': ['layer3.1.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    ### 1.1.12 - ASH MODULE AFTER layer3.1.conv2 ###
    "1.1.12.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.12.1',
        'experiment_args': {'module_placement': ['layer3.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.12.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.12.2',
        'experiment_args': {'module_placement': ['layer3.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.12.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.12.3',
        'experiment_args': {'module_placement': ['layer3.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    ### 1.1.13 - ASH MODULE AFTER layer4.0.conv1 ###
    "1.1.13.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.13.1',
        'experiment_args': {'module_placement': ['layer4.0.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.13.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.13.2',
        'experiment_args': {'module_placement': ['layer4.0.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.13.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.13.3',
        'experiment_args': {'module_placement': ['layer4.0.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    ### 1.1.14 - ASH MODULE AFTER layer4.0.conv2 ###
    "1.1.14.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.14.1',
        'experiment_args': {'module_placement': ['layer4.0.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.14.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.14.2',
        'experiment_args': {'module_placement': ['layer4.0.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.14.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.14.3',
        'experiment_args': {'module_placement': ['layer4.0.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    ### 1.1.15 - ASH MODULE AFTER layer4.1.conv1 ###
    "1.1.15.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.15.1',
        'experiment_args': {'module_placement': ['layer4.1.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.15.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.15.2',
        'experiment_args': {'module_placement': ['layer4.1.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.15.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.15.3',
        'experiment_args': {'module_placement': ['layer4.1.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    ### 1.1.16 - ASH MODULE AFTER layer4.1.conv2 ###
    "1.1.16.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.16.1',
        'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.16.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.16.2',
        'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.1.16.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.1.16.3',
        'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },

    #### 1.2 - TWO ASH MODULES ####
    ### 1.2.1 - ASH MODULE AFTER layer1.1.conv2 AND layer2.1.conv2 ###
    "1.2.1.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.2.1.1',
        'experiment_args': {'module_placement': ['layer1.1.conv2', 'layer2.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.2.1.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.2.1.2',
        'experiment_args': {'module_placement': ['layer1.1.conv2', 'layer2.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.2.1.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.2.1.3',
        'experiment_args': {'module_placement': ['layer1.1.conv2', 'layer2.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    ### 1.2.2 - ASH MODULE AFTER layer1.1.conv2 AND layer3.0.conv1 ###
    "1.2.2.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.2.2.1',
        'experiment_args': {'module_placement': ['layer1.1.conv2', 'layer3.0.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.2.2.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.2.2.2',
        'experiment_args': {'module_placement': ['layer1.1.conv2', 'layer3.0.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.2.2.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.2.2.3',
        'experiment_args': {'module_placement': ['layer1.1.conv2', 'layer3.0.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    ### 1.2.3 - ASH MODULE AFTER layer1.1.conv2 AND layer3.1.conv2 ###
    "1.2.3.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.2.3.1',
        'experiment_args': {'module_placement': ['layer1.1.conv2', 'layer3.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.2.3.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.2.3.2',
        'experiment_args': {'module_placement': ['layer1.1.conv2', 'layer3.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.2.3.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.2.3.3',
        'experiment_args': {'module_placement': ['layer1.1.conv2', 'layer3.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    ### 1.2.4 - ASH MODULE AFTER layer2.1.conv2 AND layer3.0.conv1 ###
    "1.2.4.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.2.4.1',
        'experiment_args': {'module_placement': ['layer2.1.conv2', 'layer3.0.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.2.4.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.2.4.2',
        'experiment_args': {'module_placement': ['layer2.1.conv2', 'layer3.0.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.2.4.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.2.4.3',
        'experiment_args': {'module_placement': ['layer2.1.conv2', 'layer3.0.conv1'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    ### 1.2.5 - ASH MODULE AFTER layer2.1.conv2 AND layer3.1.conv2 ###
    "1.2.5.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.2.5.1',
        'experiment_args': {'module_placement': ['layer2.1.conv2', 'layer3.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.2.5.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.2.5.2',
        'experiment_args': {'module_placement': ['layer2.1.conv2', 'layer3.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.2.5.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.2.5.3',
        'experiment_args': {'module_placement': ['layer2.1.conv2', 'layer3.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    ### 1.2.6 - ASH MODULE AFTER layer3.0.conv1 AND layer3.1.conv2 ###
    "1.2.6.1" : { # Art Painting -> Cartoon
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.2.6.1',
        'experiment_args': {'module_placement': ['layer3.0.conv1', 'layer3.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.2.6.2" : { # Art Painting -> Sketch
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.2.6.2',
        'experiment_args': {'module_placement': ['layer3.0.conv1', 'layer3.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
    "1.2.6.3" : { # Art Painting -> Photo
        'seed': 0,
        'test_only': False,
        'cpu': False,
        'experiment': 'random',
        'experiment_name': '1.2.6.3',
        'experiment_args': {'module_placement': ['layer3.0.conv1', 'layer3.1.conv2'], 'mask_out_ratio': 0.5},
        'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
        'batch_size': 128,
        'epochs': 30,
        'num_workers': 5,
        'grad_accum_steps': 1
    },
}


#### Experiment

In [21]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
# conf = configurations['0.1']
conf = configurations['1.2.1.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='1.2.1.2', experiment_args={'module_placement': ['layer1.1.conv2', 'layer2.1.conv2'], 'mask_out_ratio': 0.5}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 98.2MB/s]
  0%|          | 0/16 [00:00<?, ?it/s]

Register a hook to perform Random Maps Ablation on layer  layer1.1.conv2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
Register a hook to perform Random Maps Ablation on layer  layer2.1.conv2 Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.5
random_mask sum():  tensor(12829481., device='cuda:0')
mask_out_ratio:  0.5
random_mask sum():  tensor(6414735., device='cuda:0')
Remove the hook used to perform Activation Shaping on layer  layer1.1.conv2
Remove the hook used to perform Activation Shaping on layer  layer2.1.conv2


 12%|█▎        | 2/16 [00:08<00:52,  3.74s/it]

Register a hook to perform Random Maps Ablation on layer  layer1.1.conv2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
Register a hook to perform Random Maps Ablation on layer  layer2.1.conv2 Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.5
random_mask sum():  tensor(12833696., device='cuda:0')
mask_out_ratio:  0.5
random_mask sum():  tensor(6416454., device='cuda:0')
Remove the hook used to perform Activation Shaping on layer  layer1.1.conv2
Remove the hook used to perform Activation Shaping on layer  layer2.1.conv2


...

Register a hook to perform Random Maps Ablation on layer  layer1.1.conv2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
Register a hook to perform Random Maps Ablation on layer  layer2.1.conv2 Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.5
random_mask sum():  tensor(12836514., device='cuda:0')
mask_out_ratio:  0.5
random_mask sum():  tensor(6415147., device='cuda:0')
Remove the hook used to perform Activation Shaping on layer  layer1.1.conv2
Remove the hook used to perform Activation Shaping on layer  layer2.1.conv2
Register a hook to perform Random Maps Ablation on layer  layer1.1.conv2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
Register a hook to perform Random Maps Ablation on layer  layer2.1.conv2 Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.5
random_mask sum():  

100%|██████████| 16/16 [00:08<00:00,  1.93it/s]


tensor(12830853., device='cuda:0')
mask_out_ratio:  0.5
random_mask sum():  tensor(6412195., device='cuda:0')
Remove the hook used to perform Activation Shaping on layer  layer1.1.conv2
Remove the hook used to perform Activation Shaping on layer  layer2.1.conv2


100%|██████████| 31/31 [00:11<00:00,  2.72it/s]


Accuracy: 44.95 - Loss: 0.012841930960419525 



## 0 - Baseline

##### 0.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['0.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 0.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['0.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 0.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['0.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

## 1 - Activation Shaping module

### 1.1 - One ASH module

#### 1.1.1 - ASH module after layer1.0.conv1

##### 1.1.1.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.1.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.1.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.1.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.1.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.1.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.1.2 - ASH module after layer1.0.conv2

##### 1.1.2.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.2.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.2.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.2.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.2.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.2.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.1.3 - ASH module after layer1.1.conv1

##### 1.1.3.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.3.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.3.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.3.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.3.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.3.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.1.4 - ASH module after layer1.1.conv2

##### 1.1.4.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.4.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.4.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.4.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.4.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.4.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.1.5 - ASH module after layer2.0.conv1

##### 1.1.5.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.5.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.5.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.5.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.5.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.5.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.1.6 - ASH module after layer2.0.conv2

##### 1.1.6.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.6.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.6.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.6.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.6.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.6.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.1.7 - ASH module after layer2.1.conv1

##### 1.1.7.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.7.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.7.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.7.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.7.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.7.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.1.8 - ASH module after layer2.1.conv2

##### 1.1.8.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.8.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.8.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.8.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.8.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.8.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.1.9 - ASH module after layer3.0.conv1

##### 1.1.9.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.9.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.9.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.9.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.9.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.9.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.1.10 - ASH module after layer3.0.conv2

##### 1.1.10.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.10.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.10.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.10.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.10.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.10.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.1.11 - ASH module after layer3.1.conv1

##### 1.1.11.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.11.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.11.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.11.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.11.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.11.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.1.12 - ASH module after layer3.1.conv2

##### 1.1.12.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.12.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.12.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.12.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.12.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.12.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.1.13 - ASH module after layer4.0.conv1

##### 1.1.13.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.13.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.13.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.13.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.13.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.13.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.1.14 - ASH module after layer4.0.conv2

##### 1.1.14.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.14.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.14.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.14.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.14.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.14.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.1.15 - ASH module after layer4.1.conv1

##### 1.1.15.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.15.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.15.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.15.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.15.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.15.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.1.16 - ASH module after layer4.1.conv2

##### 1.1.16.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.16.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.16.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.16.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.1.16.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.1.16.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

### 1.2 - Two ASH modules

#### 1.2.1 - ASH module after layer1.1.conv2 and layer2.1.conv2

##### 1.2.1.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.2.1.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.2.1.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.2.1.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.2.1.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.2.1.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.2.2 - ASH module after layer1.1.conv2 and layer3.0.conv1

##### 1.2.2.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.2.2.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.2.2.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.2.2.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.2.2.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.2.2.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.2.3 - ASH module after layer1.1.conv2 and layer3.1.conv2

##### 1.2.3.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.2.3.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.2.3.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.2.3.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.2.3.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.2.3.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.2.4 - ASH module after layer2.1.conv2 and layer3.0.conv1

##### 1.2.4.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.2.4.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.2.4.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.2.4.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.2.4.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.2.4.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.2.5 - ASH module after layer2.1.conv2 and layer3.1.conv2

##### 1.2.5.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.2.5.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.2.5.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.2.5.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.2.5.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.2.5.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

#### 1.2.6 - ASH module after layer3.0.conv1 and layer3.1.conv2

##### 1.2.6.1 - Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.2.6.1']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.2.6.2 - Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.2.6.2']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

##### 1.2.6.3 - Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = configurations['1.2.6.3']

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()